## Imports

In [1]:
import numpy as np 
import pandas as pd 
import os
import gc
import copy
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import keras
from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.resnet50 import preprocess_input, decode_predictions

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Using TensorFlow backend.


## Set Data path

In [2]:
datadir = os.path.expanduser('~/Dev/humpbackwhale/data')
pretrainedmodeldir = os.path.expanduser('~/Dev/humpbackwhale/pretrainedmodels')
os.listdir(os.path.expanduser(datadir))

['trainAnalysisExcel.csv',
 '.DS_Store',
 'test',
 'data.zip',
 'trainexcel.xlsx',
 'trainAnalysisExcel.xlsx',
 'trainexcel.csv',
 'train',
 'train.csv',
 '~$trainAnalysisExcel.xlsx',
 'sample_submission.csv']

In [3]:
train_df = pd.read_csv(datadir + '/train.csv')
#train_df = train_df[:500]
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [4]:
def prepareImages(data, m, dataset):
    print("Preparing images")
    X_train = np.zeros((m, 128, 128, 3))
    count = 0
    
    for fig in data['Image']:
        #load images into images of size 100x100x3
        img = image.load_img(datadir+"/"+dataset+"/"+fig, target_size=(128, 128, 3))
        x = image.img_to_array(img)
        #x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        x = x.astype('float32')/255
        X_train[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return X_train

In [5]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded)

    y = onehot_encoded
    # print(y.shape)
    return y, label_encoder

In [6]:
X = prepareImages(train_df, train_df.shape[0], "train")
#X /= 100
X.shape

Preparing images
Processing image:  1 ,  0000e88ab.jpg
Processing image:  501 ,  04c72257b.jpg
Processing image:  1001 ,  09cacb84d.jpg
Processing image:  1501 ,  0ef961892.jpg
Processing image:  2001 ,  141b56a1a.jpg
Processing image:  2501 ,  199a417aa.jpg
Processing image:  3001 ,  1ec170983.jpg
Processing image:  3501 ,  23f084b93.jpg
Processing image:  4001 ,  29163ad0b.jpg
Processing image:  4501 ,  2e0fab120.jpg
Processing image:  5001 ,  3347515d9.jpg
Processing image:  5501 ,  3842d71dc.jpg
Processing image:  6001 ,  3d7f4c7d5.jpg
Processing image:  6501 ,  425f763ca.jpg
Processing image:  7001 ,  4714400cd.jpg
Processing image:  7501 ,  4c082fbdf.jpg
Processing image:  8001 ,  50c683e23.jpg
Processing image:  8501 ,  560d986ad.jpg
Processing image:  9001 ,  5b68c83ed.jpg
Processing image:  9501 ,  60410f111.jpg
Processing image:  10001 ,  654951f81.jpg
Processing image:  10501 ,  6a572256c.jpg
Processing image:  11001 ,  6f96f55b6.jpg
Processing image:  11501 ,  74da2b511.jpg

(25361, 128, 128, 3)

In [7]:
y, label_encoder = prepare_labels(train_df['Id'])


/Users/marcelschneider/anaconda3/envs/humpbackwhale/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [8]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [18]:
datagen = image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

### Individual Architecture neural networks

In [19]:
def CNN_Simple(): 
    model = Sequential()

    model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (128, 128, 3)))

    model.add(BatchNormalization(axis = 3, name = 'bn0'))
    model.add(Activation('relu'))

    model.add(MaxPooling2D((2, 2), name='max_pool'))
    model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
    model.add(Activation('relu'))
    model.add(AveragePooling2D((3, 3), name='avg_pool'))

    model.add(Flatten())
    model.add(Dense(500, activation="relu", name='rl'))
    model.add(Dropout(0.8))
    model.add(Dense(y.shape[1], activation='softmax', name='sm'))

    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    model.summary()
    
    return model

### Transfer Learning

In [20]:
bottleneck_features = np.load(pretrainedmodeldir+"/DogResnet50Data.npz")
train_Resnet50 = bottleneck_features['train']
test_Resnet50 = bottleneck_features['test']

def resnet_model(trainbottleneckfeatures):
    model = Sequential()
    
    model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (128, 128, 3)))

    model.add(GlobalAveragePooling2D(input_shape=trainbottleneckfeatures.shape[1:]))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(y.shape[1], activation='softmax', name='sm'))
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    model.summary()
    
    return model

In [21]:
from keras.applications import MobileNet

def mobilenet_model():
    base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

    xx=base_model.output
    xx=GlobalAveragePooling2D()(xx)
    xx=Dense(1024,activation='relu')(xx) #we add dense layers so that the model can learn more complex functions and classify for better results.
    xx=Dense(1024,activation='relu')(xx) #dense layer 2
    xx=Dense(512,activation='relu')(xx) #dense layer 3
    xx=Dropout(0.8)(xx)
    preds=Dense(y.shape[1],activation='softmax')(xx) #final layer with softmax activation
    
    model=Model(inputs=base_model.input,outputs=preds)
    #for i,layer in enumerate(model.layers):
        #print(i,layer.name)
    print(model.summary())
    # or if we want to set the first 20 layers of the network to be non-trainable
    for layer in model.layers[:50]:
        layer.trainable=False
    for layer in model.layers[50:]:
        layer.trainable=True
    
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    
    return model
              
              

In [22]:
from keras.applications.resnet50 import ResNet50, preprocess_input

def resnet50_model(): 
    base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(200, 200, 3))
    
    for layer in base_model.layers:
        layer.trainable = False
        
    x = base_model.output
    x = Flatten()(x)
    for fc in [1024,1024]:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(0.5)(x)

    # New softmax layer
    preds = Dense(y.shape[1], activation='softmax')(x) 
    
    model = Model(inputs=base_model.input, outputs=preds)
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    print(model.summary())
    
    return model

In [23]:
#from keras.applications.densenet.densenet121 import DenseNet121

def DenseNet121():
    base_model = keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet', input_shape=(100,100,3))
    
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(128, activation='relu', name='fc2')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.9, name='dropout_fc2')(x)
    preds = Dense(y.shape[1], activation='softmax')(x) 
    model = Model(inputs=base_model.input, outputs=preds)
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    print(model.summary())

    return model
    

In [24]:
#print(train_Resnet50.shape[1:])
#print(X.shape)
#print("Resnet Model: ")
#model = resnet_model(train_Resnet50)
#print("mobile Net Model: ")
#model = mobilenet_model()
#model = resnet50_model()
model = CNN_Simple()
#model = DenseNet121()

resumetraining = False
if resumetraining:
    model = load_model('historySimpleCnn.h5')



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 122, 122, 32)      4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 122, 122, 32)      128       
_________________________________________________________________
activation_3 (Activation)    (None, 122, 122, 32)      0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 61, 61, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 59, 59, 64)        18496     
_________________________________________________________________
activation_4 (Activation)    (None, 59, 59, 64)        0         
_________________________________________________________________
avg_pool (AveragePooling2D)  (None, 19, 19, 64)        0         
__________

### config of training session

In [25]:
highresimages = False
imageaugmentation = False


In [ ]:
#datagen.fit(X_train)
history_model = model.fit(X, y, epochs=10, batch_size=500, verbose=1)
#history_model = model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
#                    steps_per_epoch=len(X_train) / 100, epochs=10)

gc.collect()

Epoch 1/10
 5500/25361 [=====>........................] - ETA: 6:26 - loss: 7.2043 - acc: 0.3291

### Save trained Model

In [ ]:
from keras.models import load_model
model.save('historySimpleCnn.h5')  # creates a HDF5 file 'my_model.h5'


In [ ]:
#plt.plot(history.history['acc'])
plt.plot(history_model.history['acc'])
plt.title('Model accuracy CNN Simple with HighRes')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
#predictions = model.predict(np.array(X_test), verbose=1)
#predictions = model.predict(np.array(X_test), verbose=1)


In [ ]:
#pred_df = copy.copy(train_df)
#for i, pred in enumerate(predictions):
#    pred_df.loc[i, 'Id'] = label_encoder.inverse_transform(pred.argsort()[-5:][::-1])

In [ ]:
#print(pred_df.head(10))


### Evaluation of Prediction Precision

In [ ]:
def map_per_image(label, predictions):
    """Computes the precision score of one image.

    Parameters
    ----------
    label : string
            The true label of the image
    predictions : list
            A list of predicted elements (order does matter, 5 predictions allowed per image)

    Returns
    -------
    score : double
    """   
    score = 0.0
    try:
        if not len(np.where(predictions == label)[0])==0:
            score =  1.0/(float(np.where(predictions == label)[0]) + 1)
            return score
        else:
            return 0.
            
    except ValueError:
        return 0.0

def map_per_set(labels, predictions):
    """Computes the average over multiple images.

    Parameters
    ----------
    labels : list
             A list of the true labels. (Only one true label per images allowed!)
    predictions : list of list
             A list of predicted elements (order does matter, 5 predictions allowed per image)

    Returns
    -------
    score : double
    """
    return np.mean([map_per_image(l, p) for l,p in zip(labels, predictions)])


In [ ]:
totalaccuracy = 0.
totalaccuracy = map_per_set(train_df['Id'],pred_df['Id'])

In [ ]:
print(totalaccuracy)

### Create submission.csv (not in use for Udacity Capstone project, but created anyway)

In [ ]:
#pred_df.to_csv('submission.csv', index=False)

### Predict for Testdata

In [ ]:
test = os.listdir(datadir+"/test/")
print(len(test))

In [ ]:
col = ['Image']
test_df = pd.DataFrame(test, columns=col)
test_df['Id'] = ''

In [ ]:
X = prepareImages(test_df, test_df.shape[0], "test")
X /= 255

In [ ]:
#predictions = model.predict(np.array(X), verbose=1)
predictions = model.predict(np.array(X), verbose=1)


In [ ]:
for i, pred in enumerate(predictions):
    test_df.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))

In [ ]:
test_df.head(10)
test_df.to_csv('submission.csv', index=False)